In [1]:
import import_ipynb
from STGCN import Model
from tqdm import tqdm
import tensorflow as tf
import argparse
import inspect
import shutil
import yaml
import os
import pandas as pd

importing Jupyter notebook from STGCN.ipynb
importing Jupyter notebook from skeleton.ipynb


In [2]:
def get_parser():
    parser = argparse.ArgumentParser(
        description='Spatial Temporal Graph Convolutional Neural Network for Skeleton-Based Action Recognition')
    parser.add_argument(
        '--base-lr', type=float, default=1e-1, help='initial learning rate')
    parser.add_argument(
        '--num-classes', type=int, default=49, help='number of classes in dataset')
    parser.add_argument(
        '--batch-size', type=int, default=64, help='training batch size')
    parser.add_argument(
        '--num-epochs', type=int, default=50, help='total epochs to train')
    parser.add_argument(
        '--save-freq', type=int, default=10, help='periodicity of saving model weights')
    parser.add_argument(
        '--checkpoint-path',
        default="checkpoints",
        help='folder to store model weights')
    parser.add_argument(
        '--log-dir',
        default="logs",
        help='folder to store model-definition/training-logs/hyperparameters')
    parser.add_argument(
        '--train-data-path',
        default="tfrecord",
        help='path to folder with training dataset tfrecord files')
    parser.add_argument(
        '--test-data-path',
        default="tfrecord",
        help='path to folder with testing dataset tfrecord files')
    parser.add_argument(
        '--steps',
        type=int,
        default=[10, 40],
        nargs='+',
        help='the epoch where optimizer reduce the learning rate, eg: 10 50')
    parser.add_argument(
        '--gpus',
        default= None,
        nargs='+',
        help='list of gpus to use for training, eg: "/gpu:0" "/gpu:1"')

    return parser


In [3]:
def get_dataset(directory, num_classes=49, batch_size=32, drop_remainder=False,
                shuffle=False, shuffle_size=1000):
    # dictionary describing the features.
    feature_description = {
        'features': tf.io.FixedLenFeature([], tf.string),
        'label'     : tf.io.FixedLenFeature([], tf.int64)
    }

    # parse each proto and, the features within
    def _parse_feature_function(example_proto):
        features = tf.io.parse_single_example(example_proto, feature_description)
        data =  tf.io.parse_tensor(features['features'], tf.float32)
        index = tf.one_hot(features['label'], 2959)
        label = tf.one_hot(features['label'], num_classes)
        data = tf.reshape(data, (3, 16, 20, 1))
        return data, label, index

    records = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith("tfrecord")]
    dataset = tf.data.TFRecordDataset(records, num_parallel_reads=len(records))
    dataset = dataset.map(_parse_feature_function)
    dataset = dataset.batch(batch_size, drop_remainder=drop_remainder)
    dataset = dataset.prefetch(batch_size)
    if shuffle:
        dataset = dataset.shuffle(shuffle_size)
    return dataset

In [4]:
@tf.function
def train_step(features, labels):
    def step_fn(features, labels):
        with tf.GradientTape() as tape:
            logits = model(features, training=True)
            cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                              labels=labels)
            loss = tf.reduce_sum(cross_entropy) * (1.0 / global_batch_size)

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(list(zip(grads, model.trainable_variables)))
        train_acc(labels, logits)
        train_acc_top_5(labels, logits)
        cross_entropy_loss(loss)
    strategy.run(step_fn, args=(features, labels,))

In [5]:
@tf.function
def test_step(features):
    logits = model(features, training=False)
    return tf.nn.softmax(logits)

In [ ]:
if __name__ == "__main__":
    parser = get_parser()
    arg, unknown = parser.parse_known_args()

    base_lr         = arg.base_lr
    num_classes     = arg.num_classes
    epochs          = arg.num_epochs
    checkpoint_path = arg.checkpoint_path
    log_dir         = arg.log_dir
    train_data_path = arg.train_data_path
    test_data_path  = arg.test_data_path
    save_freq       = arg.save_freq
    steps           = arg.steps
    batch_size      = arg.batch_size
    gpus            = arg.gpus
    strategy        = tf.distribute.MirroredStrategy(arg.gpus)
    global_batch_size = arg.batch_size*strategy.num_replicas_in_sync
    arg.gpus        = strategy.num_replicas_in_sync

    #copy hyperparameters and model definition to log folder
    #save_arg(arg)
    shutil.copy2(inspect.getfile(Model), arg.log_dir)
    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

    '''
    Get tf.dataset objects for training and testing data
    Data shape: features - batch_size, 3, 300, 25, 2
                labels   - batch_size, num_classes
    '''
    train_data = get_dataset(train_data_path,
                             num_classes=num_classes,
                             batch_size=global_batch_size,
                             drop_remainder=True,
                             shuffle=True)
    
    train_data = strategy.experimental_distribute_dataset(train_data)

    test_data = get_dataset(test_data_path,
                            num_classes=num_classes,
                            batch_size=batch_size,
                            drop_remainder=False,
                            shuffle=False)

    boundaries = [(step*40000)//batch_size for step in steps]
    values = [base_lr]*(len(steps)+1)
    for i in range(1, len(steps)+1):
        values[i] *= 0.1**i
    learning_rate  = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)

    with strategy.scope():
        model        = Model(num_classes=num_classes)
        optimizer    = tf.keras.optimizers.SGD(learning_rate=learning_rate,
                                               momentum=0.9,
                                               nesterov=True)
        ckpt         = tf.train.Checkpoint(model=model, optimizer=optimizer)
        ckpt_manager = tf.train.CheckpointManager(ckpt,
                                                  checkpoint_path,
                                                  max_to_keep=5)

        # keras metrics to hold accuracies and loss
        cross_entropy_loss   = tf.keras.metrics.Mean(name='cross_entropy_loss')
        train_acc            = tf.keras.metrics.CategoricalAccuracy(name='train_acc')
        train_acc_top_5      = tf.keras.metrics.TopKCategoricalAccuracy(name='train_acc_top_5')

    epoch_test_acc       = tf.keras.metrics.CategoricalAccuracy(name='epoch_test_acc')
    epoch_test_acc_top_5 = tf.keras.metrics.TopKCategoricalAccuracy(name='epoch_test_acc_top_5')
    test_acc_top_5       = tf.keras.metrics.TopKCategoricalAccuracy(name='test_acc_top_5')
    test_acc             = tf.keras.metrics.CategoricalAccuracy(name='test_acc')
    summary_writer       = tf.summary.create_file_writer(log_dir)

    # Get 1 batch from train dataset to get graph trace of train and test functions
    
    for data in test_data:
        features, labels, index = data
        break

    # add graph of train and test functions to tensorboard graphs
    # Note:
    # graph training is True on purpose, allows tensorflow to get all the
    # variables, which is required for the first call of @tf.function function
    tf.summary.trace_on(graph=True)
    train_step(features, labels)
    with summary_writer.as_default():
        tf.summary.trace_export(name="training_trace",step=0)
    tf.summary.trace_off()

    tf.summary.trace_on(graph=True)
    test_step(features)
    with summary_writer.as_default():
        tf.summary.trace_export(name="testing_trace", step=0)
    tf.summary.trace_off()

    # start training
    train_iter = 0
    test_iter = 0
    for epoch in range(epochs):
        print("Epoch: {}".format(epoch+1))
        print("Training: ")
        with strategy.scope():
            for features, labels, index in tqdm(train_data):
                train_step(features, labels)
                with summary_writer.as_default():
                    tf.summary.scalar("cross_entropy_loss",
                                      cross_entropy_loss.result(),
                                      step=train_iter)
                    tf.summary.scalar("train_acc",
                                      train_acc.result(),
                                      step=train_iter)
                    tf.summary.scalar("train_acc_top_5",
                                      train_acc_top_5.result(),
                                      step=train_iter)
                cross_entropy_loss.reset_states()
                train_acc.reset_states()
                train_acc_top_5.reset_states()
                train_iter += 1

        print("Testing: ")
        for features, labels, index in tqdm(test_data):
            y_pred = test_step(features)
            pred_label = tf.keras.backend.eval(tf.math.argmax(y_pred, 1))
            orig_label = tf.keras.backend.eval(tf.math.argmax(labels, 1))
            print(pred_label, orig_label)
            print("test acc ", test_acc(labels, y_pred))
            print("epoch test acc ", epoch_test_acc(labels, y_pred))
            print("test acc top5", test_acc_top_5(labels, y_pred))
            print("epoch test acc top5 ",epoch_test_acc_top_5(labels, y_pred))
            with summary_writer.as_default():
                tf.summary.scalar("test_acc",
                                  test_acc.result(),
                                  step=test_iter)
                tf.summary.scalar("test_acc_top_5",
                                  test_acc_top_5.result(),
                                  step=test_iter)
            test_acc.reset_states()
            test_acc_top_5.reset_states()
            test_iter += 1
        with summary_writer.as_default():
            tf.summary.scalar("epoch_test_acc",
                              epoch_test_acc.result(),
                              step=epoch)
            tf.summary.scalar("epoch_test_acc_top_5",
                              epoch_test_acc_top_5.result(),
                              step=epoch)
        epoch_test_acc.reset_states()
        epoch_test_acc_top_5.reset_states()

        if (epoch + 1) % save_freq == 0:
            ckpt_save_path = ckpt_manager.save()
            print('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                                ckpt_save_path))

    ckpt_save_path = ckpt_manager.save()
    print('Saving final checkpoint for epoch {} at {}'.format(epochs,

                                                              ckpt_save_path))
    
    test_data = get_dataset("tfrecord_test",
                            num_classes=49,
                            batch_size=64,
                            drop_remainder=False,
                            shuffle=False)
    result=[]
    index_list=[]
    for features, labels, index in tqdm(test_data):
        pred_index = tf.keras.backend.eval(tf.math.argmax(index, 1))
        y_pred = test_step(features)
        pred_label = tf.keras.backend.eval(tf.math.argmax(y_pred, 1))
        for item in pred_label:
            result.append(item)
        for item in pred_index:
            index_list.append(item)

    
    print("----------------------------------------------")
    Id = [i for i in range (0, 2959)]
    predict = pd.DataFrame(Id, columns = ['Id'])
    result_order = [None]  * 2959
    for i, item in enumerate(index_list):
        result_order[item] = result[i]
    print(result_order)
    predict['Category'] = result_order
    print(predict)
    predict.to_csv('output.csv', index=False)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Num GPUs Available:  1
%%%%%%%%%%%%%%%%%%%%%%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/tas

146it [00:20,  7.07it/s]


Testing: 


2it [00:00, 14.97it/s]

[ 5  9 10  9 10  2 11 10 10  7 10 10  9 10  9  2  9  2  9 10 10  9 10 14
 10  9 10 10  3 10 10 14  9 14  7  9 10 10 10  9  2  5 11 10  9 10  9 11
 10 14 10  9  9 10  9  9  9  2 11  9 10  9 14  5] [13 19 25 27  4  2 40  4  9  6  4  6 18  5  8 20 27 32  1 13  1 18  8 17
  1 12  3  5  7 22  5  3  8  3  5  8  6 34 38  7  4 15  3  9  7  8  7 40
  9 14 24  4  7  4  5 10  9 32  3  1 44 25 35  5]
test acc  tf.Tensor(0.0625, shape=(), dtype=float32)
epoch test acc  tf.Tensor(0.0625, shape=(), dtype=float32)
test acc top5 tf.Tensor(0.46875, shape=(), dtype=float32)
epoch test acc top5  tf.Tensor(0.46875, shape=(), dtype=float32)
[11 10  2  5 10 10 11 11  2  5  9 10 10  3  2  2  2 10 10 10 10 10 10  9
 11  9  9 10 10 10  3 10 10  7  9 10  3  9 11 10 10  3 10  2  9  3  9 10
  2  9  9  2  9 10 10 10  9 10 10  3 11  3  2 10] [46 18  7  5 12  5 32  3  2 30 16  7  4  8  2  2 14 10  7 23 10 10 19  3
 25  7  7  8  5  9 28  7 19  5  4  4  3 12 11  6 10  6  1 26 14 26  4  6
  2 28  8 14  5 15 29 15  4  5 

...

144it [00:08, 18.04it/s]

[ 2 11  9  8 15  3 13 22  5 24 24  8 27  2  1 25  9  5  1  8  2  5  5  3
 22  2  1  1 10 10  7  5 19 20  3  7 40  9  8  5 10  5  8  3 16 20  2  3
 16 10  5  6 23 40 14 15 21  1  2  2  6 24 18  5] [ 2 25  9 21 15  6 13 22 10 39  7 19 27  2  1 17  4  5  1  8 20 13  5 33
 20  2  1  1 10 12  7 30  9 20  3  7 40  9  9  0 10  5  8  2 22 20  2  3
 39 10  5  6 23 40 14  7 21  1  2  2  6 18 18  5]
test acc  tf.Tensor(0.640625, shape=(), dtype=float32)
epoch test acc  tf.Tensor(0.7029034, shape=(), dtype=float32)
test acc top5 tf.Tensor(0.921875, shape=(), dtype=float32)
epoch test acc top5  tf.Tensor(0.92531025, shape=(), dtype=float32)
[20  9 14  3 12 23 21 23 13 18  3 19  1  5 18  5  4 16 27 10  5  3  3  5
 12 10 19  5 32 24 14 17  7  8 11 15 32 35 11  2 20 17  5 16 16 32  1  8
 14 40  6  2 10  1 10  9  9  9  8 14  5  8  3  8] [20  9 27  3 28 34 29 23 31 28  3 34  1 13  6  5  4 16 27 10  5  3  3  5
 12 10 13 18 32  1 14 17  7  8 11  6 32 35 11  2 20 17  5 16 16 17  1  4
 14 40  6  2 10  1 10 

147it [00:08, 17.81it/s]


[ 7  9  6  1 10  8  8  7 10  2  3 17 19 10 15 10  5  3  1  4  5  3  5  3
  3  5  3  7 15 26 10  2 15 33  3  9  3  9  1 11  8 17  5  8 24  3 19  9
  1 17  5 18  5 19 15  5  3 10  8  9 29  6  9 18] [ 6  4  6  1 10 12 10  7  7  2 14 27 19 10 10 28  5 48  1  4  5 14  5  3
  3 24  3  7  6 26 12  2  9 33  3  4 40  9  1 11  8 17  5 10 18  3 19  9
  1 17  5  7  5 19 15  5  3 10 22  9 29  6  9 10]
test acc  tf.Tensor(0.65625, shape=(), dtype=float32)
epoch test acc  tf.Tensor(0.70290947, shape=(), dtype=float32)
test acc top5 tf.Tensor(0.984375, shape=(), dtype=float32)
epoch test acc top5  tf.Tensor(0.92596984, shape=(), dtype=float32)
[ 3  5  2 16 18  8  9  3  8  9  8 21 23  4 13  6 10  5  2 10  9  3 10 20
 11 24  3  5 10 43  3  4  5  5  2  3 10  6 15  5 37 10 11  8 10  1 48 11
 14 21 21  8 15 26  5 12 15  3  1  8  3 15  3 42] [ 3  5  2 16 13  8  9 19  9  7  8  4 23  4 13  6 27  9  2 12  9  6 10 20
 11 24  3 18 43 43  3  4  5 10  2 25  1  6  6  5 37 10 11  8 13  1 41 11
 14 22 21  8  1 22  5 

138it [00:17,  7.98it/s]

In [ ]:
raw_dataset = tf.data.TFRecordDataset("tfrecord_test/test_data-0.tfrecord")

for raw_record in raw_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)